In [3]:
import tensorflow as tf
import os
from tensorflow import keras
os.environ['TFF_CPP_MIN_LOG_LEVEL'] = '2'

Since tensorflow 2.0 keras is integrated with it ans is the official higher level API and is essentially the go-to when building neural networks and models, and depending on the flexibility we need when creating a model we are going to use different APIs of keras.

# I - Sequential API of keras

In [4]:
from tensorflow.keras import layers

In [5]:
from tensorflow.keras.datasets import mnist

In [6]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

11501568/11490434 [==============================] - 2s 0us/step


In [8]:
print(x_train.shape) #We have 60000 images, where they are all 28 by 28 pixel in gray scal which means only one channel

(60000, 28, 28)


In [10]:
print(y_train.shape) #these are the possible outcomes

(60000,)


In [24]:
print(x_train[0:1,0:28])

[[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126
   136 175  26 166 255 247 127   0   0   0   0]
  [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253
   253 225 172 253 242 195  64   0   0   0   0]
  [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253
   251  93  82  82  56  39   0   0   0   0   0]
  [  0   0   0   0   0   0   0  18 219 253 253 2

In [30]:
x_train = x_train.reshape(-1,784).astype("float32") / 255 # -1 means keep whatever valus it is in the 60000 dimension,
#784 is 28*28 the new dimension
#gives float64 we use as type to minimize computation, divide by 255 to make the values between 0 and 1 for faster training
x_test = x_test.reshape(-1,784).astype("float32") / 255

In [31]:
print(x_train[0:1])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.   

In [32]:
#The sequential api is very convienient but not very flexible, for exemple it allows us one input mapped to one output

In [51]:
#we create a model and we give it a list of layers
#Dense : fully connected layer
model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10),
    ]
)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_19 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
#Here we tell keras how to configure the training part of the network,
#from_logits=True because we did not have a softmax activation so now with logits its going to send it to softmax first 
#and then map it to SparseCategoricalCrossentropy
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [38]:
model.fit(x_train,y_train,batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test,y_test,batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 6s - loss: 0.1868 - accuracy: 0.9436
Epoch 2/5
1875/1875 - 6s - loss: 0.0784 - accuracy: 0.9762
Epoch 3/5
1875/1875 - 6s - loss: 0.0534 - accuracy: 0.9826
Epoch 4/5
1875/1875 - 6s - loss: 0.0412 - accuracy: 0.9866
Epoch 5/5
1875/1875 - 6s - loss: 0.0312 - accuracy: 0.9898
313/313 - 1s - loss: 0.0771 - accuracy: 0.9793


[0.07713062316179276, 0.9793000221252441]

# Functional API of keras

In [50]:
inputs = keras.Input(shape=(28*28))
x = layers.Dense(512, activation='relu')(inputs)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [49]:
model.fit(x_train,y_train,batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test,y_test,batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 7s - loss: 0.1873 - accuracy: 0.9427
Epoch 2/5
1875/1875 - 7s - loss: 0.0802 - accuracy: 0.9739
Epoch 3/5
1875/1875 - 8s - loss: 0.0548 - accuracy: 0.9827
Epoch 4/5
1875/1875 - 9s - loss: 0.0399 - accuracy: 0.9869
Epoch 5/5
1875/1875 - 7s - loss: 0.0337 - accuracy: 0.9891
313/313 - 1s - loss: 0.0756 - accuracy: 0.9799


[0.07563992589712143, 0.9799000024795532]